# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from src.match_names import name_fbref_to_fpl, neutralize_name
from config import *

In [3]:
features = ['Was_home',
 'Rating_difference',
 'Value',
 'Transfers_balance',
 'Avg_FPL_points',
 'Min_2',
 'Gls_2',
 'Ast_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'FPL_points_2',
 'Min_4',
 'Gls_4',
 'Ast_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'FPL_points_4',
 'Min_30',
 'Gls_30',
 'Ast_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'FPL_points_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]

In [4]:
len(features)

38

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
X_test = df[df["Season"] == CURRENT_SEASON][features]

In [7]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Avg_FPL_points', 'DEF',
       'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GW', 'Gls_2', 'Gls_30', 'Gls_4', 'MID', 'Min_2',
       'Min_30', 'Min_4', 'Name_original', 'Opp_rating', 'Opp_score_2',
       'Opp_score_30', 'Opp_score_4', 'Opponent', 'Rating_difference',
       'Season', 'Team', 'Team_CS_2', 'Team_CS_30', 'Team_CS_4', 'Team_rating',
       'Team_result_2', 'Team_result_30', 'Team_result_4', 'Team_score_2',
       'Team_score_30', 'Team_score_4', 'Transfers_balance', 'Value',
       'Was_home', 'xA_2', 'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'xP',
       'Finished', 'Date', 'Price'],
      dtype='object')

In [8]:
X_test.columns

Index(['Was_home', 'Rating_difference', 'Value', 'Transfers_balance',
       'Avg_FPL_points', 'Min_2', 'Gls_2', 'Ast_2', 'xG_2', 'xA_2',
       'Team_CS_2', 'Team_score_2', 'Opp_score_2', 'Team_result_2',
       'FPL_points_2', 'Min_4', 'Gls_4', 'Ast_4', 'xG_4', 'xA_4', 'Team_CS_4',
       'Team_score_4', 'Opp_score_4', 'Team_result_4', 'FPL_points_4',
       'Min_30', 'Gls_30', 'Ast_30', 'xG_30', 'xA_30', 'Team_CS_30',
       'Team_score_30', 'Opp_score_30', 'Team_result_30', 'FPL_points_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [9]:
# df[df["FPL_pos"] == "GK"]

In [10]:
def new_predict(X):
    XGB_COMPONENT = 0.45
    OVERALL_COMPONENT = 0.5
    FORM_COMPONENT = 0.25
    
    model_pred = np.array(model.predict2(X))
    overall_pred = np.array([row["Avg_FPL_points"] for i, row in X.iterrows()])
    form_pred = np.array([row["FPL_points_4"] for i, row in X.iterrows()])
    
    return np.add( model_pred * XGB_COMPONENT, overall_pred * OVERALL_COMPONENT, form_pred * FORM_COMPONENT )

file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [11]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

# Predicting

In [12]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [13]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [14]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [15]:
preds = get_predictions(model, df, X_test)

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [16]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
107,Aaron Ramsdale,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,GK,50.0,4.233747
539,Alisson Ramses Becker,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,GK,55.0,4.110597
7148,José Malheiro de Sá,3.0,2023-24,Wolves,Everton,0.0,1715.631348,1699.505859,GK,50.0,3.619323
3772,Ederson Santana de Moraes,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,GK,55.0,3.570725
7005,Jordan Pickford,3.0,2023-24,Everton,Wolves,1.0,1699.505859,1715.631348,GK,45.0,3.483135
4059,Emiliano Martínez Romero,3.0,2023-24,Aston Villa,Burnley,0.0,1816.929932,1722.802979,GK,50.0,3.404224
10516,Nick Pope,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1887.064819,1943.976074,GK,55.0,3.351069
10624,Norberto Murara Neto,3.0,2023-24,Bournemouth,Tottenham,1.0,1662.536621,1817.656738,GK,45.0,2.534490
1472,Bernd Leno,3.0,2023-24,Fulham,Arsenal,0.0,1745.365723,1921.462769,GK,45.0,1.620618
12094,Sam Johnstone,3.0,2023-24,Crystal Palace,Brentford,0.0,1763.994995,1822.547119,GK,45.0,1.342418


In [17]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
1867,Bukayo Saka,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,86.0,5.257550
9978,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,MID,125.0,5.086303
4203,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,FWD,140.0,5.054771
4740,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,80.0,5.009655
1939,Callum Wilson,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1887.064819,1943.976074,FWD,80.0,4.450448
1760,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,MID,85.0,4.401895
107,Aaron Ramsdale,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,GK,50.0,4.233747
791,Andrew Robertson,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,DEF,65.0,4.232021
12630,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1832.364258,1777.551270,MID,66.0,4.157702
9261,Martin Ødegaard,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,85.0,4.112329


In [18]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
10003,Mohamed Salah,28.0,2023-24,Liverpool,Manchester City,1.0,1943.976074,2080.850586,MID,125.0,5.972085
10006,Mohamed Salah,31.0,2023-24,Liverpool,Sheffield United,1.0,1943.976074,1637.473755,MID,125.0,5.909197
9987,Mohamed Salah,12.0,2023-24,Liverpool,Brentford,1.0,1943.976074,1822.547119,MID,125.0,5.909197
10001,Mohamed Salah,26.0,2023-24,Liverpool,Luton,1.0,1943.976074,1602.327637,MID,125.0,5.909197
9997,Mohamed Salah,22.0,2023-24,Liverpool,Chelsea,1.0,1943.976074,1791.018188,MID,125.0,5.909197
9995,Mohamed Salah,20.0,2023-24,Liverpool,Newcastle Utd,1.0,1943.976074,1887.064819,MID,125.0,5.909197
9993,Mohamed Salah,18.0,2023-24,Liverpool,Arsenal,1.0,1943.976074,1921.462769,MID,125.0,5.909197
9992,Mohamed Salah,17.0,2023-24,Liverpool,Manchester Utd,1.0,1943.976074,1877.037842,MID,125.0,5.909197
10005,Mohamed Salah,30.0,2023-24,Liverpool,Brighton,1.0,1943.976074,1832.364258,MID,125.0,5.909197
9989,Mohamed Salah,14.0,2023-24,Liverpool,Fulham,1.0,1943.976074,1745.365723,MID,125.0,5.909197


In [19]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Mohamed Salah,198.120994
Erling Haaland,190.761572
Gabriel Martinelli Silva,172.477366
Bukayo Saka,169.553252
Miguel Almirón Rejala,159.760108
Andrew Robertson,159.278928
Trent Alexander-Arnold,158.328841
Callum Wilson,152.740541
Alisson Ramses Becker,152.303160


# Saving to file

In [20]:
preds.to_csv("predictions/preds_next_season.csv", index=False)

# Importance

In [21]:
# import shap
# explainer = shap.Explainer(model.predict, X_test[9000:10000])
# shap_values = explainer(X_test[9000:10000])

In [22]:
# shap.plots.bar(shap_values, max_display=15)

In [23]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[502])

In [24]:
# shap.plots.waterfall(shap_values[470])